In [ ]:
# user_id = 'notbobbobby'

# GET PLAYLISTS
# playlists = get_user_playlists(user_id, [discover_database=True])

# GET TRACKS
# tracks = get_user_tracks(user_id, playlists)

# GET FEATURES
# features = get_user_unique_tracks_and_features(user_id, tracks)

# COMBINE INTO ONE DATAFRAME
# df = pd.merge(tracks, playlists, how='outer', on=['playlist_id', 'playlist_name'])
# df = pd.merge(df, features, how='outer', on=['track_id', 'track_name'])

# Note: indieair showed missing songs, which indicates the discrepancies between
#       total_tracks of the playlists and output number of tracks

import os
import pandas as pd
import altair as alt

ROOT = os.getcwd()
def get_data(db, user, time, kind='library'):
    if db == '_history':
        path = os.path.expanduser(os.path.join(ROOT, db, user, '{}_{}_{}.json'.format(user, time, kind)))
    else:
        path = os.path.expanduser(os.path.join(ROOT, db, user, time, '{}_{}_{}.json'.format(user, time, kind)))
    return pd.read_json(path, orient='records', lines=True)

library = get_data('_db', 'notbobbobby', '2018-10-31')
discover = get_data('_db', 'discover', '2018-10-31')

history_n = get_data('_history', 'notbobbobby', '2018', kind='history')
history_d = get_data('_history', 'deedanvy', '2018', kind='history')
history_c = get_data('_history', 'c.bochulak', '2018', kind='history')
history = pd.concat([history_n, history_d, history_c])

from datetime import timedelta
history['played_at'] = history['played_at'].apply(lambda t: t - timedelta(hours=6))

In [ ]:
# Q: How many songs from my discover do I save each week?

# A: Gather the list of all the tracks, grouped by each Discover Week
#    Then, count the intersection of the library with the track ids of the Discover Week

def find_all(ids, kind):
    found = library[
        library.track_id.isin(ids) &
        library.playlist_id.isin(['saved_tracks'])] \
        .drop_duplicates(subset='track_id')
    return found[kind].tolist()

weekly_count = discover[discover.discover_id.isin(['notbobbobby'])] \
    .groupby('created_at')['track_id'] \
    .apply(list) \
    .reset_index()
weekly_count = weekly_count \
    .assign(tracks=weekly_count.track_id.apply(lambda ids: find_all(ids, 'track_name'))) \
    .assign(albums=weekly_count.track_id.apply(lambda ids: find_all(ids, 'album'))) \
    .assign(artists=weekly_count.track_id.apply(lambda ids: find_all(ids, 'artists')))
weekly_count = weekly_count \
    .assign(track_count=weekly_count.tracks.apply(lambda x: len(x))) \
    .assign(album_count=weekly_count.albums.apply(lambda x: len(x))) \
    .assign(artists_count=weekly_count.artists.apply(lambda x: len(x)))

In [ ]:
alt.Chart(weekly_count) \
    .mark_bar() \
    .encode(
        x=alt.X('created_at:T'),
        y=alt.Y(
            'track_count:Q',
            scale=alt.Scale(domain=[0, 12])),
        color=alt.Color('track_count:Q'),
        tooltip=[
            alt.Tooltip('created_at:T', format='%A, %B %e, %Y'),
            alt.Tooltip('track_count:Q', title='saved')]) \
    .properties(width=600)